c'est le modéle word2vec qu'on va utiliser avec cnn et lstm

In [ ]:
!pip install gensim
!pip install python-Levenshtein

     |████████████████████████████████| 50 kB 3.0 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=bdfc6be323a3b486703a0a971db939c6ef6da7b20668a9ea39918ac1d186386b
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [ ]:
import gensim
import pandas as pd

In [ ]:
import numpy as np #linear algebra
import pandas as pd #data preprocessing
import csv
import tensorflow as tf


importation des données


In [ ]:
train_data = pd.read_csv('/content/train.tsv.zip', sep = '\t')
train_data.drop(train_data.columns[0:2],axis=1,inplace=True)
train_data.rename(columns={'Sentiment': 'MOS','Phrase': 'Comment'}, inplace=True)

train_data =train_data.loc[:, ["MOS","Comment"]]
train_data['MOS']+=1
train_data

,MOS,Comment
0,2,A series of escapades demonstrating the adage ...
1,3,A series of escapades demonstrating the adage ...
2,3,A series
3,3,A
4,3,series
...,...,...
156055,3,Hearst 's
156056,2,forced avuncular chortles
156057,4,avuncular chortles
156058,3,avuncular


In [ ]:
df =pd.DataFrame(pd.read_excel('/content/Dataset.xlsx'))
df


,Statistiques horaires,Page,Catégorie d'appareil,Pays,Pages views,Vues uniques,Average visit time,Entries,Rebound rate,Departures (in%),Session Lenght,TimeToClickInMS,SpeedIndex,TTFB,OnLoad,FirstPaint,MOS,net_delay,latency,bandwidht,jitter,id_video,Comment
0,2020042904,/It,Desktop,Tunisia,52,1,32.769231,0,0.0,0.000000,1800,5643,9694,389,806,1800,3,1,107.93,3488,66,vid3,"For learn data science,\nIs it necessary to le..."
1,2020052016,/,Desktop,Tunisia,50,0,74.540000,0,0.0,0.000000,1650,4561,4885,163,330,3550,5,850,69.30,1813,57,vid2,my psychology lecturer recommend us to watch t...
2,2020051501,/It,Desktop,Tunisia,48,1,44.687500,0,0.0,0.000000,657,5484,3713,1588,2000,2822,4,1100,97.21,2763,44,vid3,Great video. You did great explaining this.
3,2020052814,/register,Desktop,Tunisia,44,1,44.209302,0,0.0,0.022727,1350,5119,4954,677,1506,3472,5,1500,111.86,1350,46,vid9,I recommend this video to anyone trying to los...
4,2020052101,/,Desktop,Tunisia,40,1,63.300000,1,0.0,0.000000,1030,4169,4567,117,257,0,5,2000,46.26,378,50,vid9,"I took too many breaks, lol. This is a killer ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,2020051609,/,Desktop,Tunisia,1,1,0.000000,1,1.0,1.000000,19,6361,3419,178,388,2753,3,400,56.04,577,47,vid3,how can one be passionate about data science a...
147,2020060819,/,Desktop,Tunisia,1,1,0.000000,1,1.0,1.000000,13,6035,8454,576,1005,1654,4,1500,155.04,3553,37,vid6,He's so handsome 😉
148,2020051614,/?fbclid=IwAR0H7vmH4cszLtkm44y5_fVTojqz_ICvZLy...,Desktop,Tunisia,1,1,0.000000,1,0.0,1.000000,10,4765,4165,475,899,2125,5,1800,94.15,2738,38,vid6,omg😍😍🥰
149,2020051605,/?fbclid=IwAR1MLwPxGFkizMzW3LjM_KZMMys4Bgy0IT8...,Mobile,United States,1,1,0.000000,1,1.0,1.000000,50,4853,3855,719,1420,480,3,1,56.50,2385,77,vid3,complex job


élimination des colonnes inutile pour l'analyse des sentiments

In [ ]:
df.drop(df.columns[0:16],axis=1,inplace=True)
df

,MOS,net_delay,latency,bandwidht,jitter,id_video,Comment
0,3,1,107.93,3488,66,vid3,"For learn data science,\nIs it necessary to le..."
1,5,850,69.30,1813,57,vid2,my psychology lecturer recommend us to watch t...
2,4,1100,97.21,2763,44,vid3,Great video. You did great explaining this.
3,5,1500,111.86,1350,46,vid9,I recommend this video to anyone trying to los...
4,5,2000,46.26,378,50,vid9,"I took too many breaks, lol. This is a killer ..."
...,...,...,...,...,...,...,...
146,3,400,56.04,577,47,vid3,how can one be passionate about data science a...
147,4,1500,155.04,3553,37,vid6,He's so handsome 😉
148,5,1800,94.15,2738,38,vid6,omg😍😍🥰
149,3,1,56.50,2385,77,vid3,complex job


In [ ]:
df.drop(df.columns[1:6],axis=1,inplace=True)
df

,MOS,Comment
0,3,"For learn data science,\nIs it necessary to le..."
1,5,my psychology lecturer recommend us to watch t...
2,4,Great video. You did great explaining this.
3,5,I recommend this video to anyone trying to los...
4,5,"I took too many breaks, lol. This is a killer ..."
...,...,...
146,3,how can one be passionate about data science a...
147,4,He's so handsome 😉
148,5,omg😍😍🥰
149,3,complex job


In [ ]:
frames = [df,train_data]
data = pd.concat(frames)
data

,MOS,Comment
0,3,"For learn data science,\nIs it necessary to le..."
1,5,my psychology lecturer recommend us to watch t...
2,4,Great video. You did great explaining this.
3,5,I recommend this video to anyone trying to los...
4,5,"I took too many breaks, lol. This is a killer ..."
...,...,...
156055,3,Hearst 's
156056,2,forced avuncular chortles
156057,4,avuncular chortles
156058,3,avuncular


In [ ]:
data.shape

(156211, 2)

In [ ]:
data_prep =data.Comment.apply(gensim.utils.simple_preprocess)
data_prep

0         [for, learn, data, science, is, it, necessary,...
1         [my, psychology, lecturer, recommend, us, to, ...
2         [great, video, you, did, great, explaining, this]
3         [recommend, this, video, to, anyone, trying, t...
4         [took, too, many, breaks, lol, this, is, kille...
                                ...                        
156055                                             [hearst]
156056                        [forced, avuncular, chortles]
156057                                [avuncular, chortles]
156058                                          [avuncular]
156059                                           [chortles]
Name: Comment, Length: 156211, dtype: object

construction du modéle word2vec 

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)

In [ ]:
model.build_vocab(data_prep, progress_per=1000)

In [ ]:
model.train(data_prep, total_examples=model.corpus_count, epochs=model.epochs)

(4095286, 5039805)

enregistrement du modéle pour l'utiliser avec CNN et LSTM

In [ ]:
model.save("/content/word2vec-movie_review.model")
